# Transfer Learning
The goal of this practice is to perform transefer leaning on the CIFAR-10 data set with a pretrained network GoogLeNet and some quick neural training.


## 0. Preprations

#### 0.1 Importing libraries

In [1]:
import matplotlib.pyplot as plt
from urllib.request import urlopen
import pickle
import numpy as np
import zipfile
import os
import time

from scipy.ndimage import zoom
import tensorflow as tf

#### 0.2 Tensorflow configurations

In [2]:
sess = None
def reset_tf():
    global sess
    if sess:
        sess.close()
    tf.reset_default_graph()
    sess = tf.Session()


def reset_vars():
    sess.run(tf.global_variables_initializer())

## 1. CIFAR-10 Dataset Preparation

We will be working with the _CIFAR-10_ Dataset. This dataset has been preprocessed at some levels, which worked quite well for the other questions, but not for this one. Therefore, we need perform some thansformation.

**Current Dataset**:
* train_images: (imgN=50000, height=32, width=32, channelN=3)
* validation_images: (5000, 32, 32, 3)

Format: np.float, RRB scale=(0,1)

**Target Transformation**:
* GooLeNet accommodation:
    * Zoom the image size from 32 x 32 to 288 x 288 (zoom by 7)
    * Transef the RRB scale from (0,1) to (0,255)
* Memory friendly
    * use dtype=uint8:
    * split training_images into training sets(5 batches) and one testing sets


#### Helper Functions

In [3]:
def _save_data(save_data, save_name):
    with open('{}.pkl'.format(save_name), 'wb') as f:
        pickle.dump(save_data, f)

def _load_data(load_name):
    with open('{}.pkl'.format(load_name), 'rb') as f:
        return pickle.load(f)
    
def _scale_transform(input_data):
    """
        input:   np.float, RGB scale=(0,1) 
        output:  np.unit8, RGB scale=(0,255)
    """
    output = list(map(lambda x: x * 255, input_data))
    return np.array(output).astype(np.uint8)

def _make_batch(input_features, input_labels=None, 
                batch_num=None, batch_size=None):
    input_num = len(input_features)
    
    # prerequisitions
    if all((batch_num, batch_size)):
        assert input_num == batch_size * batch_num
    elif not any((batch_num, batch_size)):
        raise NameError('Missing batch_size and batch_num')
    
    # compute batch_size if batch_num is provided
    if not batch_size:   
        batch_size = input_num // batch_num

    for start in range(0, input_num, batch_size):
        end = min(start + batch_size, input_num)
        if input_labels is None:
            yield input_features[start: end]
        else:
            yield (input_features[start: end], 
                    input_labels[start: end])

def _image_zoom(image_stack, zoom_factor):
    pass
    


In [180]:
def transform_and_save(input_features, input_labels, batch_num, save_name):
    """
    1) split the training_images into 5 batches
    2) for each batch, assign 90% of the data as training samples
    and 10% as testing samples
    3) combine all the testing samples together
                        
    """
    valid_features, valid_labels = [], []
    
    for batch_i, batch_data in \
        enumerate(_make_batch(input_features, input_labels, batch_num=batch_num)):
        
        batch_features, batch_labels = batch_data
        
        # transform
        scaled_features = _scale_transform(batch_features)
        val_idx = int(len(batch_labels) * 0.1)
       
        # save training - 90% data
        _save_data((scaled_features[:-val_idx],
                    batch_labels[:-val_idx]),
                   '{}_{}{:02d}'.format(save_name, 'train', batch_i))
        
        # combing testing - 10% data
        valid_features.extend(scaled_features[-val_idx:])
        valid_labels.extend(batch_labels[-val_idx:])
    
    #save testing
    _save_data((np.array(valid_features), np.array(valid_labels)),
               '{}_{}'.format(save_name, 'test'))

### Data tranformation

In [181]:
## settings
batch_num = 5

## load data 
train_images = np.load(open('train_images.npy', 'rb'))
train_labels = np.load(open('train_labels.npy', 'rb'))
validation_images = np.load(open('validation_images.npy', 'rb'))

## train
transform_and_save(train_images, train_labels, batch_num, 'CIFAR')

# validation
valid_data = _scale_transform(validation_images)
_save_data(valid_data, 'CIFAR_validate')

In [24]:
batch_num = 5

files = ['train{:02d}'.format(i) for i in range(batch_num)]
files.extend(('test', 'validate'))

print('CONFIRMING SAMPLE SIZES:\n')
print('{0: <8} | {1: <8} | {2: <5}'.format('data', 'features', 'labels'))
print('{}'.format('=' * 28))
for file in files:
    data = _load_data('CIFAR_{}'.format(file))
    if file == 'validate':
        print('{0: <8} | {1: <8} |'.format(file, data.shape[0]))
    else:
        print('{0: <8} | {1: <8} | {2: <5}'.format(file, data[0].shape[0], data[1].shape[0]))
    

CONFIRMING SAMPLE SIZES:

data     | features | labels
train00  | 9000     | 9000 
train01  | 9000     | 9000 
train02  | 9000     | 9000 
train03  | 9000     | 9000 
train04  | 9000     | 9000 
test     | 5000     | 5000 
validate | 10000    |


## 2. Get the output of the pre-trained network

##### load pretrained network

In [199]:
data_url = "http://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip"
data_dir = os.path.expanduser("~/inception/5h/")
file_path = os.path.join(data_dir, 'inception5h.zip')

if not os.path.exists(file_path):
    # Check if the download directory exists, otherwise create it.
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    # Download
    with open(file_path, "wb") as local_file:
        local_file.write(urlopen(data_url).read())
    # Extract
    zipfile.ZipFile(file_path, mode="r").extractall(data_dir)


In [200]:
path = os.path.join(data_dir, "tensorflow_inception_graph.pb")
with tf.gfile.FastGFile(path, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

In [201]:
with tf.Session() as sess:
    gnet_input = sess.graph.get_tensor_by_name("input:0")
    gnet_output = tf.squeeze(sess.graph.get_tensor_by_name("avgpool0:0"))

### calculate output
becuase of the large sample sizes, we need to batch this calculation

In [48]:
def _get_gnet_output(input_features, batch_size):
    output = []
    start_time = time.time()
    with tf.Session() as sess:
        gnet_input = sess.graph.get_tensor_by_name("input:0")
        gnet_output = tf.squeeze(sess.graph.get_tensor_by_name("avgpool0:0"))

        batched_data = _make_batch(input_features, batch_size=batch_size)
        for i, batch in enumerate(batched_data):
            past_time = time.time() - start_time
            print(i, end=' ')
            zoomed = zoom(batch, (1, 7, 7, 1), order = 1)
            output.extend(sess.run(gnet_output, 
                          {gnet_input: zoomed}))
    return np.array(output)
      

In [50]:
def get_latent(files):
    """
    get the latent output of GoogLeNet and save
    """
    for file in files:
        print(file)
        load_name = 'CIFAR_{}'.format(file) 
        load = _load_data(load_name)[0]
        output = _get_gnet_output(load, batch_size=50)
        save_name = 'latent_{}'.format(file)
        _save_data(output, save_name)
        print(' ')

# 
get_latent(files)

In [55]:
print('check latent output size')
for file in files:
    load = _load_data('latent_{}'.format(file))
    print('{0: <8} {1}'.format(file, load.shape))

check latent output size
train00  (9000, 1024)
train01  (9000, 1024)
train02  (9000, 1024)
train03  (9000, 1024)
train04  (9000, 1024)
test     (5000, 1024)
validate (10000, 1024)


## 3. Neuralnet Training

### Define Model Input and Output

In [202]:
reset_tf()

x = tf.placeholder(tf.float32, [None, 1024], name="latent")
y_label = tf.placeholder(tf.int64, [None,], name="labels")
training = tf.placeholder(tf.bool, name="training")

### Hyper-parameters

In [203]:
LEARNING_RATE = 0.00001
EPOCH_NUM = 50
BATCH_SIZE = 128
DROPOUT = 0.2
HIDDEN_SIZE = 512

### Build Full-connected Model

In [204]:
drop1 = tf.layers.dropout(x, DROPOUT, training=training)
hidden1 = tf.layers.dense(drop1, HIDDEN_SIZE , activation=tf.nn.relu, use_bias=True,
    kernel_initializer=tf.truncated_normal_initializer(stddev=LATENT_N ** -0.5))

drop2 = tf.layers.dropout(hidden1, DROPOUT, training=training)
hidden2 = tf.layers.dense(drop2, HIDDEN_SIZE , activation=tf.nn.relu, use_bias=True,
    kernel_initializer=tf.truncated_normal_initializer(stddev=HIDDEN_SIZE **-0.5))

drop3 = tf.layers.dropout(hidden2, DROPOUT, training=training)
hidden3 = tf.layers.dense(drop3, HIDDEN_SIZE , activation=tf.nn.relu, use_bias=True,
    kernel_initializer=tf.truncated_normal_initializer(stddev=HIDDEN_SIZE **-0.5))

drop4 = tf.layers.dropout(hidden3, DROPOUT, training=training)
y = tf.layers.dense(drop4, 10, activation=None, use_bias=True,
    kernel_initializer=tf.truncated_normal_initializer(stddev=HIDDEN_SIZE **-0.5))


## Define loss, train, and accuracy tensor/Operation
softmax = tf.nn.softmax_cross_entropy_with_logits_v2
loss = tf.reduce_mean(softmax(logits=y,
                              labels=tf.one_hot(y_label, 10)))
optimizer = tf.train.AdamOptimizer().minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), y_label), tf.float32))

### Train and Predict

##### load data

In [205]:
# training data
raw_X_train, raw_y_train = [], []
for i in range(batch_num):
    raw_X_train.extend(_load_data('latent_train{:02d}'.format(i)))
    raw_y_train.extend(_load_data('CIFAR_train{:02d}'.format(i))[1])
X_train = np.array(raw_X_train)
y_train = np.array(raw_y_train)

# testing data
X_test = _load_data('latent_test')
_, y_test = _load_data('CIFAR_test')


#### Train

In [207]:
print('Training...')

# Initializing the variables
reset_vars()

for epoch_i in range(EPOCH_NUM):
    train_acc, test_acc = [], []

    j = np.random.choice(len(y_train), len(y_train) // 2, replace=False)
    X_train_select =  X_train[j,:]
    y_train_select =  y_train[j].reshape(-1,)

    # training
    train_batches = _make_batch(X_train_select, y_train_select, batch_size = BATCH_SIZE)
    for X_train_batch, y_train_batch in train_batches: 
        opt_dict = {x: X_train_batch, y_label: y_train_batch, training: True}
        sess.run(optimizer, feed_dict=opt_dict)

    # cross_validation accuracy
    acc_dict = {x: X_test, y_label: y_test, training:False}
    test_acc = sess.run(accuracy, feed_dict=acc_dict)

    # acc_dict = {x: X_train, y_label: y_train, training:False}
    # train_acc = sess.run(accuracy, feed_dict=acc_dict)

    # output 
    print('EPOCH{}: test acc={:05f}'.format(epoch_i, test_acc))


Training...
EPOCH0: test acc=0.811200
EPOCH1: test acc=0.826200
EPOCH2: test acc=0.825600
EPOCH3: test acc=0.842800
EPOCH4: test acc=0.839600
EPOCH5: test acc=0.856800
EPOCH6: test acc=0.853400
EPOCH7: test acc=0.851400
EPOCH8: test acc=0.856200
EPOCH9: test acc=0.855600
EPOCH10: test acc=0.864200
EPOCH11: test acc=0.846400
EPOCH12: test acc=0.861000
EPOCH13: test acc=0.856600
EPOCH14: test acc=0.854400
EPOCH15: test acc=0.865200
EPOCH16: test acc=0.861000
EPOCH17: test acc=0.855600
EPOCH18: test acc=0.857200
EPOCH19: test acc=0.864000
EPOCH20: test acc=0.867000
EPOCH21: test acc=0.854800
EPOCH22: test acc=0.860800
EPOCH23: test acc=0.866600
EPOCH24: test acc=0.866000
EPOCH25: test acc=0.863600
EPOCH26: test acc=0.861600
EPOCH27: test acc=0.855200
EPOCH28: test acc=0.864800
EPOCH29: test acc=0.864800
EPOCH30: test acc=0.864600
EPOCH31: test acc=0.869800
EPOCH32: test acc=0.866800
EPOCH33: test acc=0.867400
EPOCH34: test acc=0.860000
EPOCH35: test acc=0.861000
EPOCH36: test acc=0.863600

#### Predict

In [208]:
X_validate = _load_data('latent_validate')
pred = sess.run(y, feed_dict={x: X_validate, training:False})

In [210]:
pred

array([[-10.937601  , -24.62299   ,   1.5771306 , ...,  18.127853  ,
        -29.210209  , -27.568802  ],
       [ -6.0771875 ,   9.587309  , -12.616006  , ..., -14.007324  ,
         -1.281367  ,  15.077904  ],
       [  0.45665035,  -2.5449746 ,  -1.8046563 , ...,  -1.3174105 ,
         -0.16612701,  -2.9723592 ],
       ...,
       [ -7.7910743 ,   6.1794963 , -12.122572  , ..., -10.409006  ,
         -0.57737803,  17.197119  ],
       [ -6.2540145 , -12.724137  ,   5.22315   , ...,  -0.08172381,
         -8.927856  , -12.070505  ],
       [-10.90924   ,  -9.936182  ,  -4.9890885 , ...,   2.2050624 ,
        -10.702816  , -11.3396635 ]], dtype=float32)